In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import math
import numpy as np
import collections
from numpy import linalg as la
import time
import json

# Part-1) Tweets pre-processings

In [3]:
"""
This dataset consists only of one line where all the 2399 tweets by the WHO twitter account. 
We use the json python library to load this text as a dictionary since the dataset is stored 
in JSON format. 
"""
docs_path = 'dataset_tweets_WHO.txt'
with open(docs_path) as fp:
    lines = fp.readline()
tweets = json.loads(lines)

In [4]:
print("First tweet structure:")
tweets['0']

First tweet structure:


{'created_at': 'Wed Oct 13 09:15:58 +0000 2021',
 'id': 1448215930178310144,
 'id_str': '1448215930178310144',
 'full_text': "It's International Day for Disaster Risk Reduction\n\n#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.\n\nStart learning today &amp; be #Ready4Response:\n👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS",
 'truncated': False,
 'display_text_range': [0, 274],
 'entities': {'hashtags': [{'text': 'OpenWHO', 'indices': [52, 60]},
   {'text': 'Ready4Response', 'indices': [232, 247]}],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/hBFFOF0xKL',
    'expanded_url': 'https://bit.ly/3wCa0Dr',
    'display_url': 'bit.ly/3wCa0Dr',
    'indices': [251, 274]}],
  'media': [{'id': 1448215398814560259,
    'id_str': '1448215398814560259',
    'indices': [275, 298],
    'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1448215398814560259/pu/img/0CO

In [5]:
tweets['0'].keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

In [6]:
print("First three full text tweets:\n")
for i in range(3):
    print("Tweet number",i+1,":\n")
    print(tweets[str(i)]["full_text"])
    print("\n")
    

First three full text tweets:

Tweet number 1 :

It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS


Tweet number 2 :

#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1


Tweet number 3 :

It's International Day for Disaster Risk Reduction
 
To better respond to emergencies countries must:
✅ invest in health care systems
✅ achieve gender equity
✅ protect marginalised groups
✅ ensure ready &amp; equitable access to supplies
 
A strong &amp; resilient health system is 🔑 https://t.co/5NALyjIymp




In [7]:
def process_tweet(line):
    """
    Pre-process the tweet text removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be pre-processed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line = line.lower() ## Transform in lowercase
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    line = tokenizer.tokenize(line)
    line = [word for word in line if word not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line = [stemmer.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    i = 0
    for word in line:
        if word[0:4] == 'http':
            line = line[:i]
        i+=1
    ## END CODE
    return line

In [8]:
"""
Here for every tweet we extract the 'full_text' variable since we will only 
search inside the text of each tweet. Thus, we pre-process each tweet and 
store it in a new dictionary called proc_tweets for later use
"""
proc_tweets = {}
for tweet_id, tweet in zip(tweets.keys(),tweets.values()):
    proc_tweets[int(tweet_id)] = process_tweet(tweet['full_text'])

In [9]:
print("First five processed tweets\n")
for i in range(3):
    print(proc_tweets[i])
    print("\n")
    

First five processed tweets

['intern', 'day', 'disast', 'risk', 'reduct', 'openwho', 'launch', 'multi', 'tier', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', 'amp', 'ready4respons']


['covid19', 'shown', 'health', 'emerg', 'disast', 'affect', 'entir', 'commun', 'especi', 'weak', 'health', 'system', 'vulner', 'popul', 'like', 'migrant', 'indigen', 'peopl', 'live', 'fragil', 'humanitarian', 'condit']


['intern', 'day', 'disast', 'risk', 'reduct', 'better', 'respond', 'emerg', 'countri', 'must', 'invest', 'health', 'care', 'system', 'achiev', 'gender', 'equiti', 'protect', 'marginalis', 'group', 'ensur', 'readi', 'amp', 'equit', 'access', 'suppli', 'strong', 'amp', 'resili', 'health', 'system']




Lab 2:

INDEXING:
1. Create inverted index
2. Make a proposal of 5 queries that will be used to evaluate our search engine
3. Apply a TF-IDF ranking to your results

EVALUATION:
1. Set the ground truth for each document and queryy (binary way)
2. Evaluate the algorithm with:
    2.1 Precision @K
    2.2 Average Precision @K 
    2.3 Mean Average Precision
    2.4 Mean Reciprocal Rank
    2.5 Normalizewd Discounted Cummulative Gain
3. Choose one vector rpresentation (TF-IDF or word2vec)

In [10]:
def get_tweet_info(tweet):
    Tweet = tweet['full_text']
    Username = tweet['user']['name']
    Date = tweet['created_at']
    #Hashtags = tweet['entities']['hashtags']['text']

    Hashtags = []
    hashtags_list = tweet['entities']['hashtags']
    for hashtag in hashtags_list:
        Hashtags.append(hashtag['text'])

    Likes = tweet['favorite_count']
    Retweets = tweet['retweet_count']
    Url = f"https://twitter.com/{tweet['user']['screen_name']}/status/{tweet['id_str']}"
    info = [Tweet, Username, Date, Hashtags, Likes, Retweets, Url]
    return info

In [17]:
def create_index(tweets):
    """
    Generates the index from our database to perform queries from
    
    Argument:
    tweets -- collection of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = {}
    id_index = {}
    for i in range(len(tweets)):
        tweet = tweets[str(i)]
        terms = process_tweet(tweet['full_text']) #get tweet text
        info = get_tweet_info(tweet) # get "document" info
        id_tweet = tweet['id']
        id_index[id_tweet]=info
        for term in terms: # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                
                index[term].append(id_tweet)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                index[term]= [id_tweet]#'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index                
                    
    return index, id_index

In [18]:
start_time = time.time()
index, id_index = create_index(tweets)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

Total time to create the index: 1.07 seconds


In [19]:
def search(query, index):
    query = process_tweet(query)
    first = True
    docs = {}
    for term in query:
        try:
            list_docs = index[term]
            if first:
                docs = set(list_docs)
                first = False
            else:
                docs &= set(list_docs)
            
        except:
            break
    docs=list(docs)
    return docs           

In [20]:
# def search(query, index):
#     query = process_tweet(query)
    

In [22]:
print("Insert your query:\n")
#query = input()
docs = search("covid",index)

for d_id in docs[:10]:
    print(f"{id_index[d_id]}\n")

Insert your query:

['@DrTedros "Although #VaccinEquity will help to end the pandemic, its effects will continue to be felt for many years – especially for the people who have been infected and will continue to suffer from the effects of post #COVID19 condition, also known as “long COVID”"-@DrTedros', 'World Health Organization (WHO)', 'Thu Oct 07 14:03:18 +0000 2021', ['VaccinEquity', 'COVID19'], 146, 37, 'https://twitter.com/WHO/status/1446113909480382464']

['RT @opsoms: Si está completamente vacunado 💉💉, ¿aún puede contraer COVID-19? \n\n🚨 No importa si está vacunado o si todavía está esperando, s…', 'World Health Organization (WHO)', 'Wed Oct 13 05:47:10 +0000 2021', [], 0, 43, 'https://twitter.com/WHO/status/1448163383493136385']

['RT @WHOPhilippines: Vaccines can’t stop #COVID19 alone, but by doing it all we can help protect ourselves and our loved ones against COVID-…', 'World Health Organization (WHO)', 'Mon Oct 11 04:39:10 +0000 2021', ['COVID19'], 0, 71, 'https://twitter.co

In [ ]:
docs = search("covid",index)

docs

In [ ]:
s1 = {'Python', 'Java','C++'}
s2 = {'C#', 'Java'}

s1 &= s2
s1